In [106]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


In [107]:
memory = ConversationBufferMemory(return_messages=True)

In [108]:
examples = [
    {
        "movie_title": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie_title": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "movie_title": "다크나이트",
        "answer": "🦇🃏🌃",
    },
    {
        "movie_title": "타이타닉",
        "answer": "🚢💑💔​",
    },
    {
        "movie_title": "매드맥스: 분노의 도로",
        "answer": "🚗🔥🏜️​",
    },
]

In [109]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie_title}"),
        ("ai", "{answer}"),
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [110]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영화 제목을 주면 세 개의 이모티콘으로 주제를 요약해 줍니다."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie_title}"),
    ]
)

In [111]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [112]:
chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


In [113]:
def invoke_chain(question):
    result = chain.invoke(
        {
            "movie_title": question,
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [114]:
invoke_chain("라라랜드")
invoke_chain("인셉션")
invoke_chain("아바타")
invoke_chain("아이언맨")
invoke_chain("스타워즈")

🎶🌆❤️content='🎶🌆❤️'
💤🌀🧠content='💤🌀🧠'
🌌🌿👽content='🌌🌿👽'
🤖🦸‍♂️💥content='🤖🦸\u200d♂️💥'
🌌⚔️🚀content='🌌⚔️🚀'


In [115]:
print(load_memory(""))

[HumanMessage(content='라라랜드'), AIMessage(content='🎶🌆❤️'), HumanMessage(content='인셉션'), AIMessage(content='💤🌀🧠'), HumanMessage(content='아바타'), AIMessage(content='🌌🌿👽'), HumanMessage(content='아이언맨'), AIMessage(content='🤖🦸\u200d♂️💥'), HumanMessage(content='스타워즈'), AIMessage(content='🌌⚔️🚀')]


In [116]:
invoke_chain("내가 스타워즈 전에 물어본 영화는 무엇인가요?")

"아이언맨"이었습니다.content='"아이언맨"이었습니다.'
